In [ ]:
import cv2
import torch
import numpy as np
import pandas as pd
from sort.sort import *  

In [ ]:
tracker = Sort() 

In [ ]:
# Car Detection Model
model_1 = torch.hub.load('ultralytics/yolov5', 'yolov5n', pretrained = True)

my_file = open(".../coco.txt", "r") 
data = my_file.read()
class_list = data.split("\n") 

In [ ]:
# Number Plate Detection Model
path = "../Custom Trained Weights.pt" # Path to Custom Trained YOLOv5s Weights
model_2 = torch.hub.load('ultralytics/yolov5', 'custom', path, force_reload = True)

In [ ]:
def Mouse(event, x, y, flags, param):
    if event == cv2.EVENT_MOUSEMOVE :  
        colorsBGR = [x, y]
        print(colorsBGR)

cv2.namedWindow('Mouse')
cv2.setMouseCallback('Mouse', Mouse)

roi = [ ] 

cap = cv2.VideoCapture("../sample.mp4")
count = 0
while True:
    ret, frame = cap.read()
    if not ret:
        break
    count += 1
    if count % 2 != 0:
        continue

    cv2.polylines(frame,[np.array(roi,np.int32)], True, (255,0,0), 2)
    
    cv2.imshow("Mouse", frame)
    if cv2.waitKey(1) & 0xFF == 27:
        break
    
cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)

In [ ]:
# Main (Considering a ROI)
cap = cv2.VideoCapture("../sample.mp4")
frame_count = 0
roi = []
car_id_set = set()

while True:
    ret, frame = cap.read()
    frame_count += 1
    if frame_count%4 != 0:
        continue
    
    # frame = cv2.resize(frame, (1020, 500))
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    results = model_1(frame)
    results_pd = results.pandas().xyxy[0]
    car_list = []
    for index,row in results_pd.iterrows():
        x1, y1, x2, y2, _, _, c = row
        if 'car' in c:
            if cv2.pointPolygonTest(np.array(roi,np.int32), ((x2, y2)), False) >= 0:
                car_list.append([x1, y1, x2, y2])
    
    if car_list:
        track_ids = tracker.update(np.asarray(car_list))

        plate_list = []
        for car in track_ids:
            x1, y1, x2, y2, car_id = map(int, car)
            car_frame = frame[y1:y2, x1:x2]

            if car_frame.size:
                if car_id not in car_id_set:
                    car_filename = f"Detected Car Frames/ID{car_id}.jpg"
                    cv2.imwrite(car_filename, car_frame)
                    car_id_set.add(car_id)
                
                results2 = (model_2(car_frame)).pandas().xyxy[0]
            
                if results2.size:
                    for index,row in results2.iterrows():
                        x1p = int(row[0])
                        y1p = int(row[1])
                        x2p = int(row[2])
                        y2p = int(row[3])
                        plate_frame = car_frame[y1p:y2p, x1p:x2p]
                        cv2.imshow("Plate", plate_frame)
                        output_filename = f"Detected Plate Frames/F{frame_count}_ID{car_id}.jpg"
                        cv2.imwrite(output_filename, plate_frame, [cv2.IMWRITE_JPEG_QUALITY, 100])
                
    if cv2.waitKey(1) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)

In [ ]:
# Main (Without Considering a ROI)
cap = cv2.VideoCapture("../sample.mp4")
frame_count = 0
car_id_set = set()

while True:
    ret, frame = cap.read()
    if ret is False: break
    frame_count += 1
    if frame_count%4 != 0:
        continue
    
    # frame = cv2.resize(frame, (1020, 500))
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    results = model_1(frame)
    results_pd = results.pandas().xyxy[0]
    car_list = []
    for index,row in results_pd.iterrows():
        x1, y1, x2, y2, _, _, c = row
        if 'car' in c:
            car_list.append([x1, y1, x2, y2])
    
    if car_list:
        track_ids = tracker.update(np.asarray(car_list))

        plate_list = []
        for car in track_ids:
            x1, y1, x2, y2, car_id = map(int, car)
            car_frame = frame[y1:y2, x1:x2]

            if car_frame.size:
                if car_id not in car_id_set:
                    car_filename = f"Detected Car Frames/ID{car_id}.jpg"
                    cv2.imwrite(car_filename, car_frame)
                    car_id_set.add(car_id)
                
                results2 = (model_2(car_frame)).pandas().xyxy[0]
            
                if results2.size:
                    for index,row in results2.iterrows():
                        x1p = int(row[0])
                        y1p = int(row[1])
                        x2p = int(row[2])
                        y2p = int(row[3])
                        plate_frame = car_frame[y1p:y2p, x1p:x2p]
                        cv2.imshow("Plate", plate_frame)
                        output_filename = f"Detected Plate Frames/F{frame_count}_ID{car_id}.jpg"
                        cv2.imwrite(output_filename, plate_frame, [cv2.IMWRITE_JPEG_QUALITY, 100])
                
    if cv2.waitKey(1) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)